# Store music metadata using apple shortcuts
In this small project I was trying to store the metadata about the music that gets shazamed on my macOS
I started by trying to find Shazam database file on my system but that proved to be quit tricky
Note that I tried this on MacOs, it might be different in iOS device
TODO: test on mobile device

## Create a Shortcut
the the shortcut app, click on the plus sign, a new shortcut will be created
<img src="imgs/shazam_create.png" alt="create a new shortcut" width="70%" style="margin:5px;"/>


also note that in the video I recorded I used a loop, bad idea since we can run a shortcut in CLI

I modified the short cut to perform one cycle, it takes no input and outputs a song metadata if a song is found as text, otherwise it return some "No song found"

<img src="imgs/shortcut.png" alt="shows the text file created by the shortcut app" width="80%" style="margin:5px;"/>

In [1]:
import os
import re
import pandas as pd
from bs4 import BeautifulSoup

In [11]:
SHAZAM_TEMPLATE = """
<?xml version="1.0" encoding="UTF-8"?>
<root>

<timestamp>
Date
</timestamp>

<title>
Shazam Media (Title)
</title>

<artist>
Shazam Media (Artist)
</artist>

<isexplicit>
Shazam Media (Is Explicit)
</isexplicit>

<lyricssnippet>
Shazam Media (Lyrics Snippet)
</lyricssnippet>

<lyricsnippetsynced>
Shazam Media (Lyric Snippet Synced)
</lyricsnippetsynced>

<artwork>
Shazam Media (Artwork)
</artwork>

<videourl>
Shazam Media (Video URL)
</videourl>

<shazamurl>
Shazam Media (Shazam URL)
</shazamurl>

<applemusicurl>
Shazam Media (Apple Music URL)
</applemusicurl>

<name>
Shazam Media (Name)
</name>

</root>
"""

In [19]:
outfile = './test.txt'
res = !shortcuts run "shazam_step" --output-path $outfile

In [20]:
res

[]

find the valid tags then find their values

In [41]:
def parse_row(outfile, encoding='utf-8'):
    """
    The parse_row function parses the XML document using lxml parser.
    it uses the `SHAZAM_TEMPLATE` string as map to read the xml data
    ---------------------------------------------------------
    :param outfile: Specify the path to the xml file that will be parsed
    :return: A dictionary with keys as the tag names
        and values as the text content of those tags
    """
    soup = BeautifulSoup(SHAZAM_TEMPLATE, 'xml')
    root = soup.root
    tags = [tag.name  for tag in root.children if tag and tag.name!='root']
    print()
    try:
        with open(outfile, encoding=encoding) as f:
            row = f.read()
    except FileNotFoundError:
        return FileNotFoundError
    # print(row)
    soup = BeautifulSoup(row.replace('\n', ''), 'xml')
    root = soup.select_one('root')
    if root:
        dct = {}
        for tag in tags:
            if not tag:
                continue
            tag_content = root.find(tag)
            if tag_content:
                dct[tag] = tag_content.text
    return dct

In [42]:
parse_row(outfile)

{'timestamp': '9 May 2024 at 18:57',
 'title': 'Aloha',
 'artist': 'Cho Jung-seok',
 'isexplicit': 'No',
 'lyricssnippet': '어두운 불빛 아래 촛불 하나와인 잔에 담긴 약속 하나항상 너의 곁에서 널 지켜 줄 거야날 믿어 준 너였잖아나 바라는 건 오직 하나영원한 행복을 꿈꾸지만화려하지 않아도 꿈같진 않아도너만 있어 주면 돼걱정 마 (I believe)',
 'lyricsnippetsynced': "'Cause your love is so sweetYou are my everything첫날밤의 단꿈에 젖어하는 말이 아냐 난 변하지 않아오직 너만 바라볼 거야 OhYou're light of my lifeYou are the one in my life",
 'artwork': 'Image',
 'videourl': '',
 'shazamurl': 'https://www.shazam.com/track/511213452/aloha?co=GB=shortcuts=169884=0.0010513067=244999=2024-05-09T17:57:05.276Z',
 'applemusicurl': 'https://music.apple.com/gb/album/aloha/1570389407?i=1570389411=30201=Shazam_shortcuts',
 'name': 'Cho Jung-seok - Aloha'}

In [27]:
!cat $outfile

<?xml version="1.0" encoding="UTF-8"?>
<root>

<timestamp>
9 May 2024 at 18:57
</timestamp>

<title>
Aloha
</title>

<artist>
Cho Jung-seok
</artist>

<isexplicit>
No
</isexplicit>

<lyricssnippet>
어두운 불빛 아래 촛불 하나
와인 잔에 담긴 약속 하나
항상 너의 곁에서 널 지켜 줄 거야
날 믿어 준 너였잖아
나 바라는 건 오직 하나
영원한 행복을 꿈꾸지만
화려하지 않아도 꿈같진 않아도
너만 있어 주면 돼
걱정 마 (I believe)
</lyricssnippet>

<lyricsnippetsynced>
'Cause your love is so sweet
You are my everything
첫날밤의 단꿈에 젖어
하는 말이 아냐 난 변하지 않아
오직 너만 바라볼 거야 Oh
You're light of my life
You are the one in my life
</lyricsnippetsynced>

<artwork>
Image
</artwork>

<videourl>

</videourl>

<shazamurl>
https://www.shazam.com/track/511213452/aloha?co=GB&referrer=shortcuts&offsetInMilliseconds=169884&timeSkew=0.0010513067&trackLength=244999&startDate=2024-05-09T17:57:05.276Z
</shazamurl>

<applemusicurl>
https://music.apple.com/gb/album/aloha/1570389407?i=1570389411&itscg=30201&itsct=Shazam_shortcuts
</applemusicurl>

<name>
Cho Jung-seok - Aloha
</name>

</root>